In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns

weapons = pd.read_csv("../data/510049986/intermediate_data.csv",
                      encoding='windows-1252', 
                      index_col=0)

C:\Users\Alex\Anaconda3\envs\wargame-analysis\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,2,6,10,12,13,15,16,17,18,19,20,23,24,25,26,30,31,34,40,42,43,44,45,46,47,51,55,57,58,60,61,65,66,67,68,69,70,72,73,77,81,83,84,86,87,88,89,90,91,94,95,96,97,101,102,105,111,113,114,115,116,117,118,122,126,129,131,136,137,138,139,140,141,143,144,148,152,154,155,157,158,159,160,161,162,165,166,167,168,172,173,176,182,184,185,186,187,188,189,193,197,200,202,207,208,209,210,211,212,214,215,219,223,225,226,228,229,230,231,232,236,237,238,239,243,244,247,255,256,257,258,259,260,264,268,270,271,273,274,278,279,280,281,282,283,285,286,290,294,296,297,299,300,301,302,303,307,308,309,310,314,315,318,326,327,328,329,330,331,335,339,341,342,344,345,349,350,351,352,353,354,395,420,421,423,471,492,497,499,515,521,530,531,547,564,565,568,572,591,593,603,604,621,622,625,638,641,645,656,657,660,661,665,667,668,669,671,672,673,674,675,676,677,678,679,680,681,682,688,689,692,695

In [2]:
hashes = pd.read_csv("../raws/510053208/ZZ_Win/pc/localisation/us/localisation/unites_fixed.csv", 
                     encoding='windows-1252', 
                     index_col=0)

In [3]:
units = pd.read_csv("../data/510049986/raw_data.csv", encoding="windows-1252", index_col=0)

C:\Users\Alex\Anaconda3\envs\wargame-analysis\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7,19,32,40,56,57,65,72,73,89,91,95,97,98,102,106,108,109,111,112,113,114,115,116,119,120,121,122,126,127,130,136,138,139,140,141,142,143,147,151,153,154,156,157,161,162,163,164,165,166,168,169,173,177,179,180,182,183,184,185,186,187,190,191,192,193,197,198,201,207,209,210,211,212,213,214,218,222,225,227,232,233,234,235,236,237,239,240,244,248,250,251,253,254,255,256,257,258,261,262,263,264,268,269,272,278,280,281,282,283,284,285,289,293,296,298,303,304,305,306,307,308,310,311,315,319,321,322,324,325,326,327,328,332,333,334,335,339,340,343,351,352,353,354,355,356,360,364,366,367,369,370,374,375,376,377,378,379,381,382,386,390,392,393,395,396,397,398,399,403,404,405,406,410,411,414,422,423,424,425,426,427,431,435,437,438,440,441,445,446,447,448,449,450,460,474,475,491,516,517,519,535,537,549,550,591,606,607,610,612,615,624,625,662,667,669,681,685,687,697,698,731,7

Remove deprecated units.

In [4]:
units = units[~units.apply(lambda srs: 'Deprec!' in srs['NameInMenu'], axis='columns')]
units = units[~units.apply(lambda srs: 'DEPRECATED' in srs['NameInMenu'], axis='columns')]

Merge weapons on.

In [5]:
units = units.drop([c for c in units.columns if 'Weapon' in c], axis=1).join(weapons)

In [6]:
decks = {
    '8BD43C9757360E00': 'Mechanized',
    '5C76718B57360E00': 'Armored',
    '5E767965E3000000': 'Motorized',
    'DAD77965E3000000': 'Support',
    '23B8605ED9380000': 'Marine',
    '0BB7685ED9380000': 'Airborne'
}

deck_types = units['UnitTypeTokens'].map(eval).map(lambda l: [decks[i] for i in l])
units['MechanizedDeck'] = deck_types.map(lambda t: 'Mechanized' in t)
units['ArmoredDeck'] = deck_types.map(lambda t: 'Armored' in t)
units['MotorizedDeck'] = deck_types.map(lambda t: 'Motorized' in t)
units['SupportDeck'] = deck_types.map(lambda t: 'Support' in t)
units['MarineDeck'] = deck_types.map(lambda t: 'Marine' in t)
units['AirborneDeck'] = deck_types.map(lambda t: 'Airborne' in t)

In [7]:
movers = {
    1: 'Foot',
    2: 'Wheeled',
    3: 'Wheeled',
    5: 'Tracked',
    6: 'Air',
    7: 'Wheeled',
    8: 'Tracked',
    9: 'Water'
}
amphib = {
    1: False,
    2: False,
    3: False,
    5: False,
    6: np.nan,
    7: True,
    8: True,
    9: np.nan
}

units['MovementType'] = units['UnitMovingType'].map(movers)
units['Amphibious'] = units['UnitMovingType'].map(amphib)

In [8]:
training = {
    '8F37594F19619C07' : 'Elite',
    '5593495D19619C07' : 'Shock',
    'D6173D5C19619C07' : 'Regular',
    'DE644D5719619C07' : 'Militia',
    'null': np.nan
}

units['Training'] = units['Training'].map(training)

In [9]:
ciws = {
    '4F233E0000000000': 'Exceptional',
    '4E96452000000000': 'Very Good',
    '4E96450000000000': 'Good',
    'D672711906000000': 'Medium',
    'CEC2000000000000': 'Bad',
    'null': np.nan
}

units['CIWS'] = units['CIWS'].map(ciws)

In [10]:
sailing = {
    'CBD32D65B4780000': 'Deep Sea',
    'CBD33165B4780000': 'Coastal',
    'CBD33565B4780000': 'Riverine',
    'null': np.nan
}
units['Sailing'] = units['Sailing'].map(sailing)

In [11]:
units['HitRollSizeModifier'] = units['HitRollSizeModifier'].map(lambda v: float(v) if v != "null" else np.nan)

In [12]:
mother_country = {
    'US': 'United States',
    'UK': 'United Kingdom',
    'FR': 'France',
    'RFA': 'West Germany',
    'CAN': 'Canada',
    'SWE': 'Sweden',
    'NOR': 'Norway',
    'DAN': 'Denmark',
    'ANZ': 'ANZAC',
    'JAP': 'Japan',
    'ROK': 'South Korea',
    'ISR': 'Israel',
    'HOL': 'The Netherlands',
    'URSS': 'Soviet Union',
    'RDA': 'East Germany',
    'TCH': 'Czechoslavakia',
    'POL': 'Poland',
    'CHI': 'China',
    'NK': 'North Korea'
}

units['MotherCountry'] = units['MotherCountry'].map(mother_country)

In [13]:
units['ProductionYear'] = units['ProductionYear'].map(lambda v: int(v) if v != "null" else np.nan)

In [14]:
units['MaxPacks'] = units['MaxPacks'].map(lambda v: int(v) if v != "null" else np.nan)

In [15]:
tab = {
    3: 'LOG',
    6: 'INF',
    7: 'PLA',
    8: 'VHC',
    9: 'TNK',
    10: 'REC',
    11: 'HEL',
    12: 'SHP',
    13: 'SUP',
    "null": np.nan
}
units['Tab'] = units['Factory'].map(tab)

In [16]:
units['ProductionPrice'] = units['ProductionPrice'].map(lambda l: eval(l)[0] if len(eval(l)) > 0 else np.nan)

In [17]:
units['IsPrototype'] = units['IsPrototype'].map(lambda v: True if v == "True" else False)

In [18]:
units['HitRollECMModifier'] = units['HitRollECMModifier'].map(lambda v: float(v) if v != "null" else np.nan)

In [19]:
units['Maxspeed'] = units['Maxspeed'] / 52
units['MaxSpeed'] = units['Maxspeed']
del units['Maxspeed']

In [20]:
units = units.rename(columns={'TempsDemiTour': 'TimeHalfTurn'})

In [21]:
units = units.drop(['_ShortDatabaseName', 'StickToGround', 'ManageUnitOrientation',
            'IconeType', 'PositionInMenu', 'NameInMenuToken', 'AliasName', 'Category',
            'AcknowUnitType', 'TypeForAcknow', 'Nationalite', 'Factory',
            'CoutEtoile', 'UnitTypeTokens', 'UnitMovingType', 'Key',
            'TypeUnitValue', 'UnitInfoJaugeType',
            'SpeedBonusOnRoad', 'VehicleSubType', 'TerrainsToIgnoreMask',
            'DeploymentDuration', 'WithdrawalDuration', 
            ], axis='columns')

In [22]:
deployables = units['MaxDeployableAmount'].map(lambda l: [int(v) for v in eval(l)])
units['RookieDeployableAmount'] = [d[0] for d in deployables]
units['TrainedDeployableAmount'] = [d[1] for d in deployables]
units['HardenedDeployableAmount'] = [d[2] for d in deployables]
units['VeteranDeployableAmount'] = [d[3] for d in deployables]
units['EliteDeployableAmount'] = [d[4] for d in deployables]

In [23]:
units['UnitStealthBonus'] = units['UnitStealthBonus'].map(lambda v: float(v) if v != "null" else np.nan)

In [24]:
units['AutoOrientation'] = units['AutoOrientation'].map(lambda v: True if v != "null" else False)

In [25]:
def splash(v):
    return True if 1 <= v <= 4 else False

units['ArmorFrontSplashResistant'] = units['ArmorFront'].map(splash)
units['ArmorSidesSplashResistant'] = units['ArmorSides'].map(splash)
units['ArmorRearSplashResistant'] = units['ArmorRear'].map(splash)
units['ArmorTopSplashResistant'] = units['ArmorTop'].map(splash)

In [26]:
av = lambda v: v if v <= 4 else v - 4
    
units['ArmorFront'] = units['ArmorFront'].map(av)
units['ArmorSides'] = units['ArmorSides'].map(av)
units['ArmorRear'] = units['ArmorRear'].map(av)
units['ArmorTop'] = units['ArmorTop'].map(av)

In [27]:
units['CanWinExperience'] = units['CanWinExperience'].map(lambda v: True if v != "null" else False)

In [28]:
units['IsCommandUnit'] = units['IsCommandUnit'].map(lambda v: True if v != "null" else False)

In [29]:
units['IsShip'] = units['IsTargetableAsBoat'].map(lambda v: True if v != "null" else False)
units = units.drop(['IsTargetableAsBoat'], axis='columns')

In [30]:
del units['CanWinExperience']
del units['ExperienceGainBySecond']
del units['GunMuzzleSpeed']
del units['MaxDeployableAmount']
del units['PaliersPhysicalDamages']
del units['PaliersSuppressDamages']
del units['ShowInMenu']
del units['SuppressDamagesRegenRatioOutOfRange']
del units['VitesseCombat']

In [31]:
i = 10

In [32]:
for col in ['DeathExplosionRadiusSplashSuppressDamages', 'DetectionTBA']:
    pass

In [33]:
units['LowAltitudeFlyingAltitude']= units['LowAltitudeFlyingAltitude'].map(lambda v: int(v) / 52 if v != "null" else np.nan)

In [34]:
units['NearGroundFlyingAltitude'] = units['NearGroundFlyingAltitude'].map(lambda v: int(v) / 52 if v != "null" else np.nan)

In [35]:
units = units.rename(columns={'LowAltitudeFlyingAltitude': 'HelicopterFlyingAltitude',
                              'NearGroundFlyingAltitude': 'HelicopterHoverAltitude'})

In [36]:
units['MinimalAltitude'] = units['MinimalAltitude'] / 52

In [37]:
units = units.rename(columns={'MinimalAltitude': 'AirplaneMinimalAltitude'})

In [38]:
units = units.rename(columns={'MaxDamages': 'Strength'})

In [39]:
units = units.rename(columns={'MaxSuppressionDamages': 'SuppressionCeiling'})

In [40]:
units['OpticalStrengthAntiradar'] = units['OpticalStrengthAntiradar'].map(lambda v: int(v) if v != "null" else np.nan)

In [41]:
units['PorteeVisionTBA'] = units['PorteeVisionTBA'].map(lambda v: int(v) if v != "null" else np.nan)

In [42]:
units = units.rename(columns={'ProductionPrice': 'Price'})
units = units.rename(columns={'ProductionYear': 'Year'})

In [43]:
units = units.rename(columns={'StunDamagesRegen': 'StunDamageRegen', 'StunDamagesToGetStunned': 'StunDamageToGetStunned'})

In [44]:
units['IsTransporter']= units['Transporter'].map(lambda v: True if v != "null" else False)
del units['Transporter']

In [45]:
units = units.rename(columns={'UnitStealthBonus': 'Stealth'})

In [46]:
units = units.rename(columns={'UpgradeRequired': 'Upgrade'})

In [47]:
def upgradify(srs):
    # print(srs['Upgrade'])
    if pd.isnull(srs['Upgrade']):
        return [srs['ID']]
    else:
        return [srs['ID']] + upgradify(units.query('ID == {0}'.format(srs['Upgrade'])).iloc[0])

In [48]:
all_upgrade_paths = np.unique(units.apply(upgradify, axis='columns').values)
all_upgrade_paths_sorted = sorted([sorted(l, reverse=False) for l in all_upgrade_paths], reverse=False)

In [49]:
all_full_paths = []
for i in range(len(all_upgrade_paths_sorted) - 1):
    if all_upgrade_paths_sorted[i] == []:
        pass
    else:
        flag = True
        for sublist in all_upgrade_paths_sorted[:i] + all_upgrade_paths_sorted[i + 1:]:
            if len(set(all_upgrade_paths_sorted[i]).intersection(set(sublist))) == len(set(all_upgrade_paths_sorted[i])):
                flag = False
                break
        if flag:
            all_full_paths.append(all_upgrade_paths_sorted[i])

In [50]:
upgrade_paths_lookup = dict()
for l in all_full_paths:
    for e in l:
        upgrade_paths_lookup[e] = l

In [51]:
import itertools

def transporterify(srs):
    ret = srs.copy()
    ind = 0
    if isinstance(ret['Transporters'], str):
        transporters = [int(v) for v in eval(ret['Transporters'])]
        ret['Transporters'] = list(itertools.chain.from_iterable([upgrade_paths_lookup[v] for v in transporters]))
        for i in range(len(ret['Transporters'])):
            ret['Transporter{0}ID'.format(i + 1)] = ret['Transporters'][i]
    return ret

In [52]:
units = units.apply(transporterify, axis='columns')

In [53]:
def upgradesto(uid):
    try:
        upgrade_set = upgrade_paths_lookup[uid]
        for luid in [e for e in upgrade_set if e != uid]:
            if units.query("ID == {0}".format(luid)).iloc[0]['Upgrade'] == uid:
                return luid
    except:
        pass

In [54]:
units['UpgradeTo'] = units['ID'].apply(upgradesto)
units = units.rename(columns={'Upgrade': 'UpgradeFrom'})

In [55]:
del units['DeathExplosionArme']
del units['DeathExplosionRadiusSplashPhysicalDamages']
del units['DeathExplosionRadiusSplashSuppressDamages']
del units['DeathExplosionSuppressDamages']
del units['DeathExplosionID']

In [56]:
units['DetectionTBA'] = units['DetectionTBA'].map(lambda v: v / 52)
units = units.rename(columns={'DetectionTBA': 'HelicopterDetectionRadius'})

In [57]:
units = units.rename(columns={'FuelMoveDuration': 'Autonomy'})

In [58]:
i = 0

In [59]:
units['FlyingAltitude'] = units['FlyingAltitude'] / 52

In [60]:
units = units.rename(columns={'FlyingAltitude': 'AirplaneFlyingAltitude'})

In [61]:
units['HelicopterFlyingAltitude'] = units['HelicopterFlyingAltitude'].map(lambda v: v if v == 150.0 else np.nan)

In [62]:
units['AirplaneFlyingAltitude'].value_counts()

1250.0    78
1000.0    67
500.0     51
750.0     26
1500.0     7
Name: AirplaneFlyingAltitude, dtype: int64

In [63]:
units[units['HelicopterHoverAltitude'] == 12.5]['NameInMenu']

1433    HKP 6A #reco2
Name: NameInMenu, dtype: object

In [64]:
units['MaxAcceleration'] = units['MaxAcceleration'] / 52

In [65]:
units['MaxDeceleration'] = units['MaxAcceleration'] / 52

In [66]:
del units['KillExperienceBonus']

In [67]:
units = units.rename(columns={'OpticalStrength': 'OpticalStrengthGround'})

In [68]:
units['PorteeVision'] = units['PorteeVision'] / 52

In [69]:
units = units.rename(columns={'PoteeVision': 'GroundDetectionRadius'})

In [70]:
units['PorteeVisionTBA'] = units['PorteeVisionTBA'] / 52
units = units.rename(columns={'PorteeVisionTBA': 'AirToAirHelicopterDetectionRadius'})

In [71]:
units['UpwardSpeed'] = units['UpwardSpeed'] / 52

In [72]:
del units['Transporters']

In [73]:
units = units.rename(columns={'OpticalStrengthAltitude': 'OpticalStrengthAir'})

In [74]:
def recon(v):
    if "#reco1" in v:
        return "Good"
    elif "reco2" in v:
        return "Very Good"
    elif "reco3" in v:
        return "Exceptional"
    else:
        return np.nan

In [75]:
units['Optics'] = units['NameInMenu'].map(recon)
units['NameInMenu'] = units['NameInMenu'].map(lambda n: n.replace("#command ", "")\
                                              .replace("#reco1", "")\
                                              .replace("#reco2", "")\
                                              .replace("#reco3", "")\
                                              .strip())

In [76]:
hashes = pd.read_csv("../raws/510053208/ZZ_Win/pc/localisation/us/localisation/unites_fixed.csv", 
                     encoding='windows-1252', index_col=0)

In [77]:
def hashquery(uhash):
    if pd.notnull(uhash):
        return hashes.query("Hash == '{0}'".format(uhash)).iloc[0]['String']
    else:
        return np.nan

In [78]:
for i in range(1, 12):
    units['Weapon{0}Name'.format(i)] = units['Weapon{0}Name'.format(i)].map(hashquery)
    units['Weapon{0}TypeArme'.format(i)] = units['Weapon{0}TypeArme'.format(i)].map(hashquery)
    units['Weapon{0}Caliber'.format(i)] = units['Weapon{0}Caliber'.format(i)].map(hashquery)

In [79]:
# Unfortunately our hash cut off the "," in 5,45mm rounds and 7,62mm rounds, both valid calibers. We can fix this manually.
# And others.
def fixmgcaliber(c):
    if str(c) == "7":
        return "7,62mm"
    elif str(c) == "5":
        return "5,45mm"
    elif str(c) == "12":
        return "12,7mm"
    elif str(c) == "14":
        return "14,5mm"
    elif str(c) == "4":
        return "4,73mm"
    elif str(c) == "6":
        return "6,5mm"
    elif str(c) == "7":
        return "72,5mm"
    else:
        return c

for i in range(1, 12):
     units['Weapon{0}Caliber'.format(i)] = units['Weapon{0}Caliber'.format(i)].map(fixmgcaliber)

In [80]:
def weaponify(srs):
    ret = pd.Series()
    for wi in [i for i in range(1, 12) if pd.notnull(srs['Weapon{0}Name'.format(i)])]:
        fragment = "Weapon{0}".format(wi)
        ret[fragment + 'Tags'] = []
        arme = srs[fragment + 'Arme']
        arme = int(arme) if arme != "null" else np.nan
        type_arme = srs[fragment + 'TypeArme']
        physical = srs[fragment + 'PhysicalDamages']
        physical = int(physical) if physical != "null" else np.nan
        # Determine AP.
        if arme < 3 or arme == "null":
            ret[fragment + 'AP'] = np.nan
        elif arme == 3:
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['AoE']
            ret[fragment + 'AP'] = np.nan            
        elif arme == 4:
            ret[fragment + 'AP'] = np.nan
        elif type_arme == "SSM":
            ret[fragment + 'AP'] = (arme - 34) * physical
        elif 5 <= arme <= 34:
            ret[fragment + 'AP'] = arme - 4
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['AoE'] + ['KE']
        elif 35 <= arme <= 65:
            ret[fragment + 'AP'] = arme - 34
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['HEAT']
        
        # Determine HE.
        if physical == "null":
            ret[fragment + 'HE'] = np.nan
        else:
            ret[fragment + 'HE'] = physical
        
        # Conversions
        for c in ['RadiusSplashPhysicalDamages', 'Puissance', 'TempsEntreDeuxTirs', 'TempsEntreDeuxSalves',
                  'NbrProjectilesSimultanes', 'NbTirParSalves', 'RadiusSplashSuppressDamages', 'TempsDeVisee',
                  'DispersionAtMaxRange', 'DispersionAtMinRange', 'CorrectedShotDispersionMultiplier']:
            if srs[fragment + c] == "null":
                srs[fragment + c] = np.nan
            else:
                srs[fragment + c] = float(srs[fragment + c])
            
        ret[fragment + 'RadiusSplashPhysicalDamage'] = srs[fragment + 'RadiusSplashPhysicalDamages'] * 175 / 13000
        ret[fragment + 'Noise'] = srs[fragment + 'Puissance']
        ret[fragment + 'TimeBetweenShots'] = srs[fragment + 'TempsEntreDeuxTirs']
        ret[fragment + 'TimeBetweenSalvos'] = srs[fragment + 'TempsEntreDeuxSalves']
        ret[fragment + 'ProjectilesPerShot'] = srs[fragment + 'NbrProjectilesSimultanes']
        ret[fragment + 'ShotsPerSalvo'] = srs[fragment + 'NbTirParSalves']
        ret[fragment + 'RadiusSplashSuppressDamage'] = srs[fragment + 'RadiusSplashSuppressDamages'] * 175 / 13000
        ret[fragment + 'AimTime'] = srs[fragment + 'TempsDeVisee']
        ret[fragment + 'DispersionAtMaxRange'] = srs[fragment + 'DispersionAtMaxRange'] * 175 / 13000
        ret[fragment + 'DispersionAtMinRange'] = srs[fragment + 'DispersionAtMinRange'] * 175 / 13000

        # Distance convesions
        for c in ['PorteeMaximale', 'PorteeMinimale', 'PorteeMinimaleBateaux', 'PorteeMaximaleBateaux',
                  'PorteeMaximaleTBA', 'PorteeMinimaleTBA', 'PorteeMinimaleHA', 'PorteeMaximaleHA',
                  'PorteeMaximaleProjectile', "PorteeMinimaleProjectile"]:
            if srs[fragment + c] == "null":
                srs[fragment + c] = np.nan
            else:
                srs[fragment + c] = float(srs[fragment + c])
        ret[fragment + 'RangeGround'] = srs[fragment + 'PorteeMaximale'] * 175 / 13000
        ret[fragment + 'RangeGroundMinimum'] = srs[fragment + 'PorteeMinimale'] * 175 / 13000
        ret[fragment + 'RangeShip'] = srs[fragment + 'PorteeMaximaleBateaux'] * 175 / 13000
        ret[fragment + 'RangeShipMinimum'] = srs[fragment + 'PorteeMinimaleBateaux'] * 175 / 13000
        ret[fragment + 'RangeHelicopters'] = srs[fragment + 'PorteeMaximaleTBA'] * 175 / 13000
        ret[fragment + 'RangeHelicoptersMinimum'] = srs[fragment + 'PorteeMinimaleTBA'] * 175 / 13000
        ret[fragment + 'RangePlanes'] = srs[fragment + 'PorteeMaximaleHA'] * 175 / 13000
        ret[fragment + 'RangePlanesMinimum'] = srs[fragment + 'PorteeMinimaleHA'] * 175 / 13000
        ret[fragment + 'RangeMissiles'] = srs[fragment + 'PorteeMaximaleProjectile'] * 175 / 13000
        ret[fragment + 'RangeMissilesMinimum'] = srs[fragment + 'PorteeMinimaleProjectile'] * 175 / 13000
        
        # Level
        lvl = srs[fragment + 'Level']
        if lvl == "null":
            ret[fragment + 'PositionOnCard'] = np.nan
        else:
            ret[fragment + 'PositionOnCard'] = int(lvl)
        
        # Guidance
        guid = srs[fragment + "Guidance"]
        if str(guid) == 1:
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['RAD']
        if str(guid) == 2:
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['SEAD']
        
        # Accuracies
        for c in ['HitProbability', 'HitProbabilityWhileMoving', 'MinimalCritProbability',
                  'MinimalHitProbability']:
            if srs[fragment + c] == "null":
                ret[fragment + c] = np.nan
            else:
                ret[fragment + c] = srs[fragment + c]
        
        # Lone tags...
        if type_arme == "SAW":
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['CQC']
        if pd.notnull(ret[fragment + 'RangeMissiles']):
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['DEF']
        if srs[fragment + 'TirIndirect'] != "null":
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['CORR']
        if srs[fragment + 'IsFireAndForget'] != "null":
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['F&F']
        if srs[fragment + 'IgnoreInflammabilityConditions'] != "null":
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['NPLM']
        if srs[fragment + 'IsSubAmmunition'] != "null":
            ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['CLUS']

# Because I attached CanSmoke incorrectly this will have to be done using a far more tedious process.
#         if fragment + 'CanSmoke' in srs and srs[fragment + 'CanSmoke'] == True:
#             ret[fragment + 'Tags'] = ret[fragment + 'Tags'] + ['SMK']
        
        # Direct carry-overs:
        for field in ["AngleDispersion", "RayonPinned", "FireTriggeringProbability", "SupplyCost",
                      "MissileTimeBetweenCorrections", "CorrectedShotDispersionMultiplier"]:
            ret[fragment + field] = srs[fragment + field]
    return ret

Next steps:

* Do missiles.
* Do the rest of the tags.
* Map tags and transporters to delimited lists.
* Recheck everything.
* Move 4,5 to a script.
* Execute the script.
* Write the data dictionary.
* Edit the README.
* Upload.

In [81]:
weaponify(units.query('NameInMenu == "ONDAVA"').iloc[0])

Weapon1Tags                                 [AoE, CORR]
Weapon1AP                                           NaN
Weapon1HE                                             7
Weapon1RadiusSplashPhysicalDamage                 106.4
Weapon1Noise                                        200
Weapon1TimeBetweenShots                             8.5
Weapon1TimeBetweenSalvos                             30
Weapon1ProjectilesPerShot                             1
Weapon1ShotsPerSalvo                                 10
Weapon1RadiusSplashSuppressDamage                 212.8
Weapon1AimTime                                       10
Weapon1DispersionAtMaxRange                         140
Weapon1DispersionAtMinRange                         140
Weapon1RangeGround                                32200
Weapon1RangeGroundMinimum                          3220
Weapon1RangeShip                                  32200
Weapon1RangeShipMinimum                            3220
Weapon1RangeHelicopters                         

In [82]:
weaponify(units.query('NameInMenu == "DELTA FORCE"').iloc[0])

Weapon1Tags                                   []
Weapon1HE                                      1
Weapon1RadiusSplashPhysicalDamage           2.45
Weapon1Noise                                  10
Weapon1TimeBetweenShots                        1
Weapon1TimeBetweenSalvos                       2
Weapon1ProjectilesPerShot                      1
Weapon1ShotsPerSalvo                           8
Weapon1RadiusSplashSuppressDamage           8.05
Weapon1AimTime                               0.2
Weapon1DispersionAtMaxRange                  NaN
Weapon1DispersionAtMinRange                  NaN
Weapon1RangeGround                           455
Weapon1RangeGroundMinimum                   1.75
Weapon1RangeShip                             455
Weapon1RangeShipMinimum                     1.75
Weapon1RangeHelicopters                      455
Weapon1RangeHelicoptersMinimum               NaN
Weapon1RangePlanes                           NaN
Weapon1RangePlanesMinimum                    NaN
Weapon1RangeMissiles

In [83]:
units.query('NameInMenu == "DELTA FORCE"').iloc[0][['Weapon1Arme', 'Weapon1PhysicalDamages', 'Weapon1TypeArme', 'Weapon1Puissance']]

Weapon1Arme               null
Weapon1PhysicalDamages       1
Weapon1TypeArme            SMG
Weapon1Puissance            10
Name: 286, dtype: object

In [84]:
units.query('NameInMenu == "SH-60B ASM"').iloc[0][['Weapon1Arme', 'Weapon1PhysicalDamages', 'Weapon1TypeArme']]

Weapon1Arme                54
Weapon1PhysicalDamages      3
Weapon1TypeArme           SSM
Name: 1546, dtype: object

In [85]:
units['Weapon1TypeName']

0       0B06000000000000
1       0B06000000000000
2       0B06000000000000
3       0B06000000000000
4       0B06000000000000
5       0B06000000000000
6       0B06000000000000
7       0B06000000000000
8       0B06000000000000
9       0B06000000000000
10      0B06000000000000
11      0B06000000000000
12      0B06000000000000
13      0B06000000000000
14      0B06000000000000
15      0B06000000000000
17      0B06000000000000
18      0B06000000000000
19      0B06000000000000
20      0B06000000000000
21      0B06000000000000
22      0B06000000000000
23      0B06000000000000
24      0B06000000000000
25      0B06000000000000
26      0B06000000000000
27      0B06000000000000
28      0B06000000000000
29      0B06000000000000
30      0B06000000000000
              ...       
1787    0B06000000000000
1788    0B06000000000000
1789    0B06000000000000
1790    0B06000000000000
1791    0B06000000000000
1792    0B06000000000000
1793    0B06000000000000
1794    0B06000000000000
1795    0B06000000000000


In [86]:
# Rename DeathExplosionSuppressDamages, DetectionTBA

In [87]:
# TODO: Map Salves, map SalvoIsMainSalvo, delete HasMainSalvo after.
# TODO: All the weapons stuff.
# TODO: Who knows how many other variables?
# TODO: Create a data dictionary.
# TODO: Turn this notebook into a repeatable script.
# TODO: Publish.
# TODO: Run on previous versions of the game.